<a href="https://colab.research.google.com/github/MikeSoricelli/Density_aware_Uncertainty_Pruning/blob/main/density_based_uncertainty_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install normflows

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.8 MB/s eta 0:00:00
  Created wheel for normflows: filename=normflows-1.7.3-py2.py3-none-any.whl siz

In [ ]:
!pip install ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.3 MB/s eta 0:00:00


In [ ]:
#import libraries

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, datasets
from torch.utils.data import DataLoader
import numpy as np
import normflows as nf
import ipdb
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader

class NoisyCIFAR10(Dataset):
    def __init__(self, root, train=True, download=True, noise_std=0.35, transform=None):
        # Load CIFAR-10 dataset
        self.base_dataset = datasets.CIFAR10(root=root, train=train, download=download, transform=transform)
        self.noise_std = noise_std
        self.transform = transform

        # Decide which indices will be noisy (random half)
        self.is_noisy = np.zeros(len(self.base_dataset), dtype=bool)
        noisy_indices = np.random.choice(len(self.base_dataset), size=len(self.base_dataset)//2, replace=False)
        self.is_noisy[noisy_indices] = True

    def __getitem__(self, idx):
        img, label = self.base_dataset[idx]
        # img is a PIL Image if no transform, or a Tensor if transform includes ToTensor
        if self.is_noisy[idx]:
            if isinstance(img, torch.Tensor):
                noise = torch.randn_like(img) * self.noise_std
                img = torch.clamp(img + noise, 0., 1.)
            else:
                # If still PIL, convert to tensor, add noise, then back to PIL if needed
                img = transforms.ToTensor()(img)
                noise = torch.randn_like(img) * self.noise_std
                img = torch.clamp(img + noise, 0., 1.)
                if self.transform is not None and not any(isinstance(t, transforms.ToTensor) for t in self.transform.transforms):
                    img = transforms.ToPILImage()(img)
        return img, label

    def __len__(self):
        return len(self.base_dataset)

class NoisyCIFAR100(Dataset):
    def __init__(self, root, train=True, download=True, noise_std=0.35, transform=None):
        # Load CIFAR-10 dataset
        self.base_dataset = datasets.CIFAR100(root=root, train=train, download=download, transform=transform)
        self.noise_std = noise_std
        self.transform = transform

        # Decide which indices will be noisy (random half)
        self.is_noisy = np.zeros(len(self.base_dataset), dtype=bool)
        noisy_indices = np.random.choice(len(self.base_dataset), size=len(self.base_dataset)//2, replace=False)
        self.is_noisy[noisy_indices] = True

    def __getitem__(self, idx):
        img, label = self.base_dataset[idx]
        # img is a PIL Image if no transform, or a Tensor if transform includes ToTensor
        if self.is_noisy[idx]:
            if isinstance(img, torch.Tensor):
                noise = torch.randn_like(img) * self.noise_std
                img = torch.clamp(img + noise, 0., 1.)
            else:
                # If still PIL, convert to tensor, add noise, then back to PIL if needed
                img = transforms.ToTensor()(img)
                noise = torch.randn_like(img) * self.noise_std
                img = torch.clamp(img + noise, 0., 1.)
                if self.transform is not None and not any(isinstance(t, transforms.ToTensor) for t in self.transform.transforms):
                    img = transforms.ToPILImage()(img)
        return img, label

    def __len__(self):
        return len(self.base_dataset)

In [ ]:
len(dataset)

60000

In [ ]:
import torch
import torch.nn as nn

class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        return self.relu2(out) + x # ReLU can be applied before or after adding the input

def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
              nn.BatchNorm2d(out_channels),
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes=10, input_size=32):
        super().__init__()

        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))

        self.classifier = nn.Linear(512, num_classes)
        self.fc1 = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                        nn.Flatten(),
                                        )

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.fc1(out)
        out = self.classifier(out)
        return out

import torch
import torch.nn as nn

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out


In [ ]:
def calculate_variance(predictions, start_window, end_window):
    predictions = predictions[start_window:end_window]
    predictions = np.array(predictions)
    variance_predictions = np.var(predictions, axis=0)
    return np.mean(variance_predictions)

In [ ]:
## dynUnc implementation https://arxiv.org/pdf/2306.05175

def compute_dyn_unc(probabilities, window_size, total_epochs):
    uncertainties = [0] * len(trainset)
    for epoch in range(total_epochs):
        if epoch >= window_size:
            # window_predictions = get_predictions(epoch-window_size, epoch)
            # uncertainties.append(calculate_variance(window_predictions))
            # predictions = probabilities[epoch-window_size:epoch]

            #for each data point calculate the uncertainty at this position and append it to the uncertainbty over all of training
            for i in range(len(trainset)):
                uncertainties[i] += calculate_variance(probabilties[i], epoch - window_size, epoch)
        else:
            uncertainties.append(0.0)
        print(epoch)
    uncertainties = np.array(uncertainties)
    return uncertainties / (total_epochs - window_size)


In [ ]:
#prune dataset

def prune_dataset(dataset, ratio, model, uncertainties, probabilities, beta):

    #normalize unceratatinties and probabiliteis

    uncertainties = np.array(uncertainties)
    probabilities = np.array(probabilities)

    #manually get min for probabitlies due to underflow
    prob_min = 0
    for i in range(len(probabilities)):
        if probabilities[i] != float('-inf') and probabilities[i] < prob_min:
            prob_min = probabilities[i]


    for i in range(len(probabilities)):
        if probabilities[i] == float('-inf'):
            probabilities[i] = prob_min


    uncertainties = (uncertainties - np.min(uncertainties)) / (np.max(uncertainties) - np.min(uncertainties))
    probabilities = (probabilities - np.min(probabilities)) / (np.max(probabilities) - np.min(probabilities))


    # Balance standard deviations
    prob_std = probabilities.std()
    uncert_std = uncertainties.std()
    uncertainties = uncertainties * (prob_std / uncert_std)

    scores = []
    for i in range(len(dataset)):
        scores.append((beta * uncertainties[i]) + ((1 - beta) * (probabilities[i])))
    threshold = np.percentile(scores, 100*(1-ratio))

    indices = []

    for i in range(len(scores)):
        if scores[i] >= threshold:
            indices.append(i)

    return [dataset[i] for i in indices], indices


#CIFAR 100 Test

In [ ]:
#flow model definition

# Define flows
L = 3
K = 16
torch.manual_seed(0)

input_shape = (3, 32, 32)
n_dims = np.prod(input_shape)
channels = 3
hidden_channels = 256
split_mode = 'channel'
scale = True
num_classes = 100

# Set up flows, distributions and merge operations
q0 = []
merges = []
flows = []
for i in range(L):
    flows_ = []
    for j in range(K):
        flows_ += [nf.flows.GlowBlock(channels * 2 ** (L + 1 - i), hidden_channels,
                                     split_mode=split_mode, scale=scale)]
    flows_ += [nf.flows.Squeeze()]
    flows += [flows_]
    if i > 0:
        merges += [nf.flows.Merge()]
        latent_shape = (input_shape[0] * 2 ** (L - i), input_shape[1] // 2 ** (L - i),
                        input_shape[2] // 2 ** (L - i))
    else:
        latent_shape = (input_shape[0] * 2 ** (L + 1), input_shape[1] // 2 ** L,
                        input_shape[2] // 2 ** L)
    q0 += [nf.distributions.ClassCondDiagGaussian(latent_shape, num_classes)]


# Construct flow model with the multiscale architecture
flow_model = nf.MultiscaleFlow(q0, flows, merges)

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
flow_model = flow_model.to(device)

/usr/local/lib/python3.11/dist-packages/torch/_tensor.py:902: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a future PyTorch release.
LU, pivots = torch.lu(A, compute_pivots)
should be replaced with
LU, pivots = torch.linalg.lu_factor(A, compute_pivots)
and
LU, pivots, info = torch.lu(A, compute_pivots, get_infos=True)
should be replaced with
LU, pivots, info = torch.linalg.lu_factor_ex(A, compute_pivots) (Triggered internally at /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2053.)
  LU, pivots, infos = torch._lu_with_info(


In [ ]:
#setup up cifar 100 dataset

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
])

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))])

trainset = NoisyCIFAR100(train=True, download=True, root='./data', noise_std=0.25, transform=transform_train)
testset = datasets.CIFAR100(train=False, download=True, root='./data', transform=transform)

trainLoader = DataLoader(trainset, batch_size=256, num_workers=4, shuffle=False)
testLoader = DataLoader(testset, batch_size=256, num_workers=4, shuffle=False)


100%|██████████| 169M/169M [00:13<00:00, 12.9MB/s]


In [ ]:
#intilaize model and run trianig loop, stroe probablities at eahc epoch for teh dataset

#data structure for trainng probabilties

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ResNet9(3, 100, 32).to(device)

batch_size = 128

model.to(device)

epoch = 50


probabilties = [[] for i in range(len(trainset))]


optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(epoch):
    model.train()
    idx = 0
    for batch_idx, (inputs, labels) in enumerate(trainLoader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track probabilities using absolute indices
        start_idx = batch_idx * trainLoader.batch_size
        for i, output in enumerate(outputs.detach().cpu().numpy()):
            probabilties[start_idx + i].append(output)  # Matches dataset order

    model.eval()
    correct = 0
    for image, label in testLoader:
        image = image.to(device)
        label = label.to(device)
        output = model(image)
        predictions = torch.argmax(output, dim=1)
        correct += torch.sum((predictions == label))
    print(f'epoch = {epoch}, test accuracy = {correct/len(testset)}')



epoch = 0, test accuracy = 0.16210000216960907
epoch = 1, test accuracy = 0.23589999973773956
epoch = 2, test accuracy = 0.3046000003814697
epoch = 3, test accuracy = 0.3741999864578247
epoch = 4, test accuracy = 0.4153999984264374
epoch = 5, test accuracy = 0.4486999809741974
epoch = 6, test accuracy = 0.4339999854564667
epoch = 7, test accuracy = 0.48339998722076416
epoch = 8, test accuracy = 0.4885999858379364
epoch = 9, test accuracy = 0.491599977016449
epoch = 10, test accuracy = 0.5194000005722046
epoch = 11, test accuracy = 0.5277000069618225
epoch = 12, test accuracy = 0.5317000150680542
epoch = 13, test accuracy = 0.5525999665260315
epoch = 14, test accuracy = 0.5425999760627747
epoch = 15, test accuracy = 0.538599967956543
epoch = 16, test accuracy = 0.5555999875068665
epoch = 17, test accuracy = 0.5565999746322632
epoch = 18, test accuracy = 0.5537999868392944
epoch = 19, test accuracy = 0.5500999689102173
epoch = 20, test accuracy = 0.5758000016212463
epoch = 21, test accur

In [ ]:
uncertainties = compute_dyn_unc(probabilties, 10, 50)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [ ]:
#load flow model

from google.colab import drive
drive.mount('/content/drive')

# 1. Instantiate your NICE model
# Construct flow model with the multiscale architecture
# 2. Load the state dictionary into the model
state_dict = torch.load('/content/drive/MyDrive/flowModels/Glow_CIFAR100_Clean.pth', map_location=torch.device('cuda'))
flow_model.load_state_dict(state_dict)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<All keys matched successfully>

In [ ]:
#generate log probabilites of each dta point:

probabilities = []

max_float32 = np.finfo(np.float32).max




with torch.no_grad():
    for image, targets in trainLoader:
        image = image.to(device)
        log_prob = flow_model.log_prob(image, targets.to(device))
        prob = torch.exp(log_prob)
        print(targets)
        for i in range(len(image)):
            print(log_prob[i].item())
            probabilities.append(log_prob[i].item())

In [ ]:
(uncertainties - np.min(uncertainties)) / (np.max(uncertainties) - np.min(uncertainties))

array([0.2170527 , 0.29429865, 0.1799405 , ..., 0.        , 0.        ,
       0.        ])

In [ ]:

values = (probabilities - np.min(probabilities)) / (np.max(probabilities) - np.min(probabilities))

values.min()

<ipython-input-14-3ee390340665>:1: RuntimeWarning: invalid value encountered in subtract
  values = (probabilities - np.min(probabilities)) / (np.max(probabilities) - np.min(probabilities))
<ipython-input-14-3ee390340665>:1: RuntimeWarning: invalid value encountered in divide
  values = (probabilities - np.min(probabilities)) / (np.max(probabilities) - np.min(probabilities))


np.float64(nan)

In [ ]:
pruned_dataset = prune_dataset(trainset, 0.5, model, uncertainties, probabilities, 1)

In [ ]:
len(pruned_dataset)

25000

In [ ]:
prune_trainLoader = DataLoader(pruned_dataset, batch_size=256, num_workers=4, shuffle=False)

In [ ]:
correct = 0
for inputs, labels in testLoader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    predictions = torch.argmax(outputs, dim=1)
    correct += torch.sum((predictions == labels))
print(correct/len(testset))

tensor(0.4701, device='cuda:0')


In [ ]:
#train model on pruned dataset

ratios = [1, 0.5]

sizes = [0.5, 0.25]

for k in sizes:

  print(f'size: {k}')


  for i in ratios:

      print(f'ratio {i}')

      accuracies = [0.0] * 9
      ratio_idx = 0
      for j in range(2):

          max_accuracy = 0


          pruned_dataset = prune_dataset(trainset, k, model, uncertainties, probabilities, i)

          prune_trainLoader = DataLoader(pruned_dataset, batch_size=256, num_workers=4, shuffle=False)

          pruned_model = ResNet9(3, 100, 32)

          pruned_model.to(device)

          epoch = 50

          optimizer = optim.Adam(pruned_model.parameters(), lr=0.001)
          criterion = nn.CrossEntropyLoss()

          for epoch in range(epoch):
              idx = 0
              for batch_idx, (inputs, labels) in enumerate(prune_trainLoader):
                  inputs, labels = inputs.to(device), labels.to(device)

                  # Forward pass
                  outputs = pruned_model(inputs)
                  loss = criterion(outputs, labels)

                  # Backward pass
                  optimizer.zero_grad()
                  loss.backward()
                  optimizer.step()

              correct = 0
              for inputs, labels in testLoader:
                  inputs, labels = inputs.to(device), labels.to(device)
                  outputs = pruned_model(inputs)
                  predictions = torch.argmax(outputs, dim=1)
                  correct += torch.sum((predictions == labels))
              # print(f'epoch:{epoch}, accuracy:{correct/len(testset)}')
              if correct/len(testset) > max_accuracy:
                  max_accuracy = correct/len(testset)

              accuracies[ratio_idx] += correct/len(testset)

          print(f'max accuracy: {max_accuracy}')

      accuracies[ratio_idx] /= 5
      ratio_idx += 1
      print(accuracies)


size: 0.5
ratio 1
max accuracy: 0.5094999670982361
max accuracy: 0.5121999979019165
[tensor(9.2641, device='cuda:0'), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ratio 0.5
max accuracy: 0.5288999676704407
max accuracy: 0.5223999619483948
[tensor(9.2942, device='cuda:0'), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
size: 0.25
ratio 1
max accuracy: 0.44119998812675476
max accuracy: 0.43529999256134033
[tensor(7.6588, device='cuda:0'), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ratio 0.5
max accuracy: 0.4527999758720398
max accuracy: 0.445499986410141
[tensor(7.6458, device='cuda:0'), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


size: 0.75

just dynunc
epoch:99, accuracy:0.3545999825000763
epoch:99, accuracy:0.3580999970436096
epoch:99, accuracy:0.3580999970436096
epoch:99, accuracy:0.3612000048160553

ratio 0.25

epoch:99, accuracy:0.33390000462532043
epoch:99, accuracy:0.3407999873161316
epoch:99, accuracy:0.3303000032901764

ratio 0.5
epoch:99, accuracy:0.343999981880188
epoch:99, accuracy:0.336899995803833


ratio 0.75


size 0.25:

ratio 1:

epoch:99, accuracy:0.24169999361038208
epoch:99, accuracy:0.2410999983549118


epoch:0, accuracy:0.18400000035762787
epoch:10, accuracy:0.29819998145103455
epoch:20, accuracy:0.33709999918937683
epoch:30, accuracy:0.346699982881546
epoch:40, accuracy:0.36319997906684875
epoch:50, accuracy:0.36409997940063477
epoch:60, accuracy:0.36629998683929443
epoch:70, accuracy:0.36550000309944153
epoch:80, accuracy:0.3651999831199646
epoch:90, accuracy:0.36569997668266296

#CIFAR10 Test

In [ ]:
#flow model definition

# Define flows
L = 3
K = 16
torch.manual_seed(0)

input_shape = (3, 32, 32)
n_dims = np.prod(input_shape)
channels = 3
hidden_channels = 256
split_mode = 'channel'
scale = True
num_classes = 10

# Set up flows, distributions and merge operations
q0 = []
merges = []
flows = []
for i in range(L):
    flows_ = []
    for j in range(K):
        flows_ += [nf.flows.GlowBlock(channels * 2 ** (L + 1 - i), hidden_channels,
                                     split_mode=split_mode, scale=scale)]
    flows_ += [nf.flows.Squeeze()]
    flows += [flows_]
    if i > 0:
        merges += [nf.flows.Merge()]
        latent_shape = (input_shape[0] * 2 ** (L - i), input_shape[1] // 2 ** (L - i),
                        input_shape[2] // 2 ** (L - i))
    else:
        latent_shape = (input_shape[0] * 2 ** (L + 1), input_shape[1] // 2 ** L,
                        input_shape[2] // 2 ** L)
    q0 += [nf.distributions.ClassCondDiagGaussian(latent_shape, num_classes)]


# Construct flow model with the multiscale architecture
flow_model = nf.MultiscaleFlow(q0, flows, merges)

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
flow_model = flow_model.to(device)

In [ ]:
#setup up cifar 100 dataset

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = NoisyCIFAR10(root='./data', train=True, download=True, noise_std = 0.5, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainLoader = DataLoader(trainset, batch_size=256, num_workers=4, shuffle=False)
testLoader = DataLoader(testset, batch_size=256, num_workers=4, shuffle=False)

In [ ]:
#intilaize model and run trianig loop, stroe probablities at eahc epoch for teh dataset

#data structure for trainng probabilties

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ResNet9(3, 10, 32).to(device)

batch_size = 128

model.to(device)


epoch = 50


probabilties = [[] for i in range(len(trainset))]


optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(epoch):
    idx = 0
    for batch_idx, (inputs, labels) in enumerate(trainLoader):
        inputs, labels = inputs.to(device), labels.to(device)

        model.train()
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track probabilities using absolute indices
        start_idx = batch_idx * trainLoader.batch_size
        for i, output in enumerate(outputs.detach().cpu().numpy()):
            probabilties[start_idx + i].append(output)  # Matches dataset order

    model.eval()
    correct = 0
    for inputs, labels in testLoader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=1)
        correct += torch.sum((predictions == labels))
    print(f'accuracy = {correct/len(testset)}')

    print(epoch)



accuracy = 0.561199963092804
0
accuracy = 0.6022999882698059
1
accuracy = 0.6714999675750732
2
accuracy = 0.7281999588012695
3
accuracy = 0.7579999566078186
4
accuracy = 0.7439999580383301
5
accuracy = 0.7421999573707581
6
accuracy = 0.7242000102996826
7
accuracy = 0.7013999819755554
8
accuracy = 0.6881999969482422
9
accuracy = 0.7674999833106995
10
accuracy = 0.7860999703407288
11
accuracy = 0.7913999557495117
12
accuracy = 0.7347999811172485
13
accuracy = 0.7724999785423279
14
accuracy = 0.78329998254776
15
accuracy = 0.7335000038146973
16
accuracy = 0.7764999866485596
17
accuracy = 0.7859999537467957
18
accuracy = 0.7976999878883362
19
accuracy = 0.7644000053405762
20
accuracy = 0.7785999774932861
21
accuracy = 0.7953999638557434
22
accuracy = 0.7935000061988831
23
accuracy = 0.7719999551773071
24
accuracy = 0.7953000068664551
25
accuracy = 0.8108999729156494
26
accuracy = 0.7906000018119812
27
accuracy = 0.7946999669075012
28
accuracy = 0.7864999771118164
29
accuracy = 0.8133999705

In [ ]:
uncertainties = compute_dyn_unc(probabilties, 10, 50)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [ ]:
#load flow model

from google.colab import drive
drive.mount('/content/drive')

# 1. Instantiate your NICE model
# Construct flow model with the multiscale architecture
# 2. Load the state dictionary into the model
state_dict = torch.load('/content/drive/MyDrive/flowModels/GLOW_CIFAR10_Clean.pth', map_location=torch.device('cuda'))
flow_model.load_state_dict(state_dict)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<All keys matched successfully>

In [ ]:
#generate log probabilites of each dta point:

probabilities = []

max_float32 = np.finfo(np.float32).max

batch_size = 128

with torch.no_grad():
    for image, targets in trainLoader:
        image = image.to(device)
        log_prob = flow_model.log_prob(image, targets.to(device))
        prob = torch.exp(log_prob)
        print(targets)
        for i in range(len(image)):
            print(log_prob[i].item())
            probabilities.append(log_prob[i].item())

In [ ]:
#train model on pruned dataset

ratios = [1, 0.5]

sizes = [0.5, 0.25]

for k in sizes:

    for i in ratios:

        print(f'size: {k}')


        print(f'ratio {i}')

        accuracies = [0.0] * 9
        ratio_idx = 0
        for j in range(2):


            pruned_dataset = prune_dataset(trainset, k, model, uncertainties, probabilities, i)

            prune_trainLoader = DataLoader(pruned_dataset, batch_size=256, num_workers=4, shuffle=False)

            pruned_model = ResNet9(3, 10, 32).to(device)

            pruned_model.to(device)

            epoch = 50

            optimizer = optim.Adam(pruned_model.parameters(), lr=0.001)
            criterion = nn.CrossEntropyLoss()

            max_accuracy = 0

            for epoch in range(epoch):
                idx = 0
                for batch_idx, (inputs, labels) in enumerate(prune_trainLoader):
                    pruned_model.train()
                    inputs, labels = inputs.to(device), labels.to(device)

                    # Forward pass
                    outputs = pruned_model(inputs)
                    loss = criterion(outputs, labels)

                    # Backward pass
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                pruned_model.eval()
                correct = 0
                for inputs, labels in testLoader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = pruned_model(inputs)
                    predictions = torch.argmax(outputs, dim=1)
                    correct += torch.sum((predictions == labels))
                if correct/len(testset) > max_accuracy:
                    max_accuracy = correct/len(testset)
            print(f'max accuracy: {max_accuracy}')

    print(accuracies)


size: 0.5
ratio 1
max accuracy: 0.8112999796867371
max accuracy: 0.8100999593734741
size: 0.5
ratio 0.5
max accuracy: 0.8129000067710876
max accuracy: 0.8126999735832214
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
size: 0.25
ratio 1
max accuracy: 0.7356999516487122
max accuracy: 0.7414999604225159
size: 0.25
ratio 0.5
max accuracy: 0.7572000026702881
max accuracy: 0.7526999711990356
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


#comparative tests


In [ ]:
import numpy as np

def random_pruning(dataset, prune_fraction):
    n = len(dataset)
    keep_n = int(n * (1 - prune_fraction))
    indices = np.random.choice(n, keep_n, replace=False)
    pruned_dataset = [dataset[i] for i in indices]
    return pruned_dataset

In [ ]:
random_prune_set = random_pruning(trainset, 0.5)

In [ ]:
import torch

def compute_grand_scores(model, dataloader, device):
    model.eval()
    scores = []
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        inputs.requires_grad = True
        outputs = model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs, targets, reduction='none')
        for i in range(len(inputs)):
            model.zero_grad()
            loss[i].backward(retain_graph=True)
            grad_norm = 0.0
            for param in model.parameters():
                if param.grad is not None:
                    grad_norm += param.grad.norm().item() ** 2
            grad_norm = grad_norm ** 0.5
            scores.append(grad_norm)
    return scores

def grand_pruning(dataset, scores, prune_fraction):
    n = len(scores)
    keep_n = int(n * (1 - prune_fraction))
    top_indices = np.argsort(scores)[-keep_n:]
    pruned_dataset = [dataset[i] for i in top_indices]
    return pruned_dataset

In [ ]:
import numpy as np


def loss_monotonicity_pruning(model, dataloader, device, epochs):
    # Initialize a history of losses for each sample
    loss_history = np.zeros((len(dataloader.dataset), epochs))
    for epoch in range(epochs):
        model.train()
        for idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = torch.nn.functional.cross_entropy(outputs, targets, reduction='none')
            loss_history[idx, epoch] = loss.item()
    # Count how many times loss increases for each sample
    loss_increases = np.sum(np.diff(loss_history, axis=1) > 0, axis=1)
    return loss_increases

def prune_by_loss_monotonicity(dataset, loss_increases, prune_fraction):
    n = len(loss_increases)
    keep_n = int(n * (1 - prune_fraction))
    top_indices = np.argsort(loss_increases)[-keep_n:]
    pruned_dataset = [dataset[i] for i in top_indices]
    return pruned_dataset



In [ ]:
print(len(gradn_prune_set))

25000


In [ ]:
random_prune_set = random_pruning(trainset, 0.75)

gradn_scores = compute_grand_scores(model, trainLoader, device)

gradn_prune_set = grand_pruning(trainset, gradn_scores, 0.5)

model = ResNet9(3, 10, 32).to(device)


pruned_sets = [random_prune_set, gradn_prune_set]

print("trainign sets computed")

for pruned_set in pruned_sets:

  prune_model = ResNet9(3, 10, 32).to(device)

  optimizer = torch.optim.Adam(prune_model.parameters(), lr=0.001)
  criterion = nn.CrossEntropyLoss()

  epochs = 50



  prune_loader = DataLoader(pruned_set, batch_size=256, num_workers=4, shuffle=False)

  max_accuracy = 0

  for epoch in range(epochs):
      for batch_idx, (inputs, labels) in enumerate(prune_loader):
          prune_model.train()
          inputs = inputs.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = prune_model(inputs)
          loss = criterion(outputs, labels)

          # Backward pass
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

      correct = 0
      for inputs, labels in testLoader:
          prune_model.eval()
          inputs = inputs.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = prune_model(inputs)

          predictions = torch.argmax(outputs, dim=1)
          correct += torch.sum((predictions == labels))
      accuracy = correct / len(testset)
      print(f'epoch: {epoch}, accuracy: {accuracy}')
      if accuracy > max_accuracy:
          max_accuracy = accuracy
  print(f'max accuracy: {max_accuracy}')

          # Backward pass

trainign sets computed
epoch: 0, accuracy: 0.35899999737739563
epoch: 1, accuracy: 0.46069997549057007
epoch: 2, accuracy: 0.49859997630119324
epoch: 3, accuracy: 0.5593000054359436
epoch: 4, accuracy: 0.583899974822998
epoch: 5, accuracy: 0.5801999568939209
epoch: 6, accuracy: 0.6111999750137329
epoch: 7, accuracy: 0.6380999684333801
epoch: 8, accuracy: 0.5575000047683716
epoch: 9, accuracy: 0.40209999680519104
epoch: 10, accuracy: 0.4779999852180481
epoch: 11, accuracy: 0.5722000002861023
epoch: 12, accuracy: 0.5543000102043152
epoch: 13, accuracy: 0.5889999866485596
epoch: 14, accuracy: 0.5482999682426453
epoch: 15, accuracy: 0.46379998326301575
epoch: 16, accuracy: 0.4197999835014343
epoch: 17, accuracy: 0.6513999700546265
epoch: 18, accuracy: 0.5496000051498413
epoch: 19, accuracy: 0.5738999843597412
epoch: 20, accuracy: 0.6354999542236328
epoch: 21, accuracy: 0.6380999684333801
epoch: 22, accuracy: 0.6847000122070312
epoch: 23, accuracy: 0.7051999568939209
epoch: 24, accuracy: 0.